In [1]:
import sys

sys.path.append('.')

import model

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import random
import seaborn as sns
import statistics

Some invariant parameters for this notebook:

In [3]:
## Population parameters:
base_params = {
    # Node parameter
    'A' : 0.0, # Now this will vary case by case.
    
    # Edge parameter
    'W' : .5, # probability of edge activation; 2/K
    'C' : 1.0, ## all edges can be traced.
    
    ## Disease parameters

    'beta_hat' : .4, # probability of transmission upon contact
    'alpha' : .25, # probability of exposed becoming infectious
    'gamma' : .1, # probability of infectious becoming recovered
    'zeta' : .1, # probability of infectious becoming symptomatic

    ## Contact tracing parameters

    'limit' : 10, # number of time steps the contact tracing system remembers
}

$p^*$ has been chosen because it exposes the effects of the Watts-Strogatz structure.
With $K = 4$, roughly one edge for every node is rewired, with the rest comprising the circle lattice.

In [4]:
p_star = 0.256
K = 4

# N = 2000  --- allowing this to vary now

This time, run the model with no contact tracing at all.

In [5]:
def watts_strogatz_case_p_star(N, K, p_star, **kwargs):

    g = nx.watts_strogatz_graph(N, K, p_star)
    
    g.graph['p'] = p_star
    
    return g, kwargs

def ws_case_generator(N, K, p_star):
    def wscg(**kwargs):
        return watts_strogatz_case_p_star(N, K, p_star, **kwargs)
    
    return wscg

In [6]:
conditions = {
    'A-0.000' : {'A' : 0.000},
    'A-0.100' : {'A' : 0.100},
    'A-0.200' : {'A' : 0.200},
    'A-0.300' : {'A' : 0.300},
    'A-0.400' : {'A' : 0.400},
    'A-0.500' : {'A' : 0.500},
    'A-0.600' : {'A' : 0.600},
    'A-0.700' : {'A' : 0.700},
    'A-0.800' : {'A' : 0.800},
    'A-0.900' : {'A' : 0.900},
    'A-1.000' : {'A' : 1.000},
}

In [7]:
def data_from_result(results, case):
    a_key = list(results.keys())[0]
    g = results[a_key][0][2]
    N = len(g.nodes)
    
    return [{**r[1],
             **{
                 "N" : N,
                 "case" : case,
                 "A" : r[1]['A'],
                 "time" : r[0],
                 "s_final" : r[4][-1],
                 "infected_ratio" : (N - r[4][-1]) / N
             }} 
            for r
            in results[case]]

def data_from_all_results(results):
    return pd.DataFrame([r for case in results for r in data_from_result(results, case)])


In [8]:
runs = 30

In [9]:
#results_2000 = model.experiment(
#    ws_case_generator(1000, K, p_star),
#    base_params,
#    conditions,
#    runs)

#data_2000 = data_from_all_results(results_2000)

#data_2000.to_csv('data_A_2000.csv')

In [10]:
#results_4000 = model.experiment(
#    ws_case_generator(2000, K, p_star),
#    base_params,
#    conditions,
#    runs)

#data_4000 = data_from_all_results(results_4000)
#data_4000.to_csv('data_A_4000.csv')

In [ ]:
results_6000 = model.experiment(
    ws_case_generator(3000, K, p_star),
    base_params,
    conditions,
    runs)

data_6000 = data_from_all_results(results_6000)
data_6000.to_csv('data_A_6000.csv')

A-0.000
Trial 0
Trial 0 hits time step 100
Trial 10 hits time step 100
Trial 13 hits time step 100
Trial 0 hits time step 200
Trial 16 hits time step 100
Trial 10 hits time step 200
Trial 13 hits time step 200
Trial 0 hits time step 300
Trial 16 hits time step 200
Trial 19 hits time step 100
Trial 21 hits time step 100
Trial 24 hits time step 100
Trial 17 hits time step 100
Trial 19 hits time step 200
Trial 21 hits time step 200
Trial 24 hits time step 200
Trial 19 hits time step 300
Trial 17 hits time step 200
Trial 21 hits time step 300
Trial 24 hits time step 300
Trial 17 hits time step 300
Trial 27 hits time step 100
Trial 25 hits time step 100
Trial 28 hits time step 100
Trial 27 hits time step 200
Trial 25 hits time step 200
Trial 27 hits time step 300
Trial 29 hits time step 100
Trial 29 hits time step 200
A-0.100
Trial 0
Trial 2 hits time step 100
Trial 11 hits time step 100
Trial 2 hits time step 200
Trial 22 hits time step 100
Trial 25 hits time step 100
Trial 11 hits time st

In [ ]:
data = pd.concat([
    pd.read_csv('data_A_2000.csv'),
    pd.read_csv('data_A_4000.csv'),
    pd.read_csv('data_A_6000.csv')
])

In [ ]:
g = results_2000['A-0.000'][0][2]
N = len(g.nodes())
bins = np.linspace(0, 1, 50)

for case in results_2000:
    plt.hist(
        [len(model.susceptible(r[2])) / N
         for r
         in results_2000[case]],
        bins,
        alpha=.5,
        label=case)
    
plt.legend()

In [ ]:
g = results_2000['A-0.000'][0][2]
bins = np.linspace(0, len(g.nodes()), 50)

for case in results_2000:
    plt.plot(
        model.average_over_time(results_2000[case], infected=True) / N,
        alpha=.8,
        label=case)
    
plt.legend()

In [ ]:
data["N-cat"] = data["N"].apply(lambda x: f"N = {x}")

splot = sns.lineplot(x='A', y='infected_ratio', hue="N-cat", data=data)

splot.set(#xscale="log",
          xlabel='adoption rate',
          ylabel='average final infected ratio')

In [ ]:
data["N"].unique()

## Finding the inflection point

Trying to find the inflection point. (What if there isn't one?)

In [ ]:
epidemic_size = data[data["N"] == 2000].groupby("A")['infected_ratio'].mean()

x = epidemic_size.index
y = epidemic_size.values

In [ ]:
df1 = np.gradient(epidemic_size.values,
            epidemic_size.index,
            edge_order = 1)

In [ ]:
np.gradient(df1, epidemic_size.index, edge_order = 1)

In [ ]:
x[np.argsort(df1)]

In [ ]:
model.inflection_point(x,y)

In [ ]:
plt.plot(x, y)


In [ ]:
plt.plot(x, df1)
